# 2020-nCoV Global Cases by Wooil Jeong

- **Wooil Jeong**  
[Dashboard by WooilJeong](https://plot.ly/dashboard/coronavirus:34/present#/)  
[Blog](https://wooiljeong.github.io/etc/corona_dash/)  
[Github Repository](https://github.com/WooilJeong/novel_coronavirus)  


- **Novel Coronavirus (2019-nCoV) Cases, provided by JHU CSSE**  
[Dashboard by JHU CSSE](https://gisanddata.maps.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6)  
[Old Data Sheets](https://docs.google.com/spreadsheets/d/1yZv9w9zRKwrGTaR-YzmAqMefw4wMlaXocejdxZaTs6w/htmlview?usp=sharing&sle=true#)  
[New Google Sheet Link (support comments)](https://docs.google.com/spreadsheets/d/1wQVypefm946ch4XDp37uZ-wartW4V7ILdg-qYiDXUHM/edit?usp=sharing)  
[Time series google sheet](https://docs.google.com/spreadsheets/d/1UF2pSkFTURko2OvfHWWlFpDFAr1UxCBA4JLwlSP6KFo/edit?usp=sharing)  


- **Contact**  
email : wooil@kakao.com  

## Create Visualization

## Libraries

In [2]:
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chart_studio
chart_studio.tools.set_credentials_file(username='coronavirus', api_key='rTk5dPkR3nQRjwyNyRfF')
import chart_studio.plotly as py
import plotly.express as px

## Load Dataset

### 프로젝트 디렉토리로 설정

In [3]:
# os.chdir('/Users/wooil/github/novel_coronavirus/')
os.getcwd()

'D:\\github\\novel_coronavirus'

### 데이터가 저장된 디렉토리 내 파일 목록 조회

In [4]:
file_list = glob.glob("Data/*")
file_list = sorted(file_list, reverse=True)
file_list

['Data\\Dataset.csv']

### 최신 데이터 불러오기

In [5]:
try:
    df = pd.read_csv(file_list[0])
except:
    df = pd.read_csv(file_list[0].replace('\\','/')+'.csv')

## 전 세계 통합 데이터

In [6]:
def agg(df):
    '''
    국가 및 지역 별 데이터 집계 함수
    '''

    df_region = df.groupby('Last Update').agg(
        {'Confirmed':['sum'], 'Death':['sum'],'Recovered':['sum']}).reset_index()
    
    df_region.index = range(len(df_region))
    df_region.columns=['Last Update', 'Confirmed', 'Death', 'Recovered']
    
    # Feature Engineering
    df_region['D/C'] = (df_region['Death']/df_region['Confirmed'])*100
    df_region['R/C'] = (df_region['Recovered']/df_region['Confirmed'])*100
    
    return df_region

In [7]:
df_all = agg(df)
df_all.tail()

,Last Update,Confirmed,Death,Recovered,D/C,R/C
19,2020-01-30 11:00:00,8235,171,141,2.076503,1.712204
20,2020-01-31 14:00:00,9877,213,220,2.156525,2.227397
21,2020-01-31 19:00:00,11373,259,248,2.277323,2.180603
22,2020-02-01 10:00:00,12024,259,284,2.154025,2.361943
23,2020-02-02 21:00:00,17295,362,487,2.093090,2.815843


## 전 세계 통합 확진자, 사망자, 회복자 추이

In [8]:
data = df_all.copy()
data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])

fig = px.line(data, x="Last Update", y="Confirmed", template='plotly_dark', title='Total Confirmed', text='Confirmed')
fig2= px.line(data_melt, x="Last Update", y="value", color='variable', template='plotly_dark', title='Total Death and Recovered', text='value')

In [9]:
fig.update_traces(textposition='top center')
py.iplot(fig, filename='corona_confirmed')

In [10]:
fig2.update_traces(textposition='top center')
py.iplot(fig2, filename='corona_death_recovered')

## 국가 및 지역 별 데이터프레임 생성

### 리스트 생성

In [11]:
# 모든 국가 리스트
country_list = list(set(df['Country/Region']))

# 제외할 국가 리스트(제외 국가는 세부 지역 별 리스트 생성하여 분석)
except_list = ['Mainland China', 'US', 'Canada', 'Australia']

# 세계 국가 리스트 재정의(일부 국가 제외)
world_list = country_list.copy()
for i in except_list:
    world_list.remove(i)

# 일부 국가(세부 지역 별 리스트 생성)
## 중국 지역 리스트
china_list = list(set(df[df['Country/Region']=='Mainland China']['Province/State']))
### 후베이 제외 중국 지역 리스트
china_list2 = china_list.copy()
china_list2.remove('Hubei')

## 미국 지역 리스트
us_list = list(set(df[df['Country/Region']=='US']['Province/State']))

## 캐나다 지역 리스트
canada_list = list(set(df[df['Country/Region']=='Canada']['Province/State']))

## 호주 지역 리스트
australia_list = list(set(df[df['Country/Region']=='Australia']['Province/State']))

### 데이터프레임 생성

In [12]:
for i in country_list:
    globals()[i.replace(' ','_')] = df[df['Country/Region']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))
     
for i in china_list:
    globals()[i.replace(' ','_')] = Mainland_China[Mainland_China['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))
    
for i in us_list:
    globals()[i.replace(' ','_')] = US[US['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))
    
for i in australia_list:
    globals()[i.replace(' ','_')] = Australia[Australia['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))
    
for i in canada_list:
    globals()[i.replace(' ','_')] = Canada[Canada['Province/State']==i]
    globals()[i.replace(' ','_')] = globals()[i.replace(' ','_')].sort_values('Last Update', ascending=True)
    globals()[i.replace(' ','_')].index = range(len(globals()[i.replace(' ','_')]))    
    print(i.replace(' ','_'))

Malaysia
Hong_Kong
Australia
Mainland_China
Japan
Germany
Nepal
UK
Macau
US
United_Arab_Emirates
Italy
India
Thailand
Cambodia
Sri_Lanka
Vietnam
South_Korea
Singapore
France
Sweden
Canada
Taiwan
Spain
Philippines
Russia
Finland
Boston,_MA
California
Washington
Arizona
Illinois
Victoria
Queensland
New_South_Wales
South_Australia
Ontario
British_Columbia


In [13]:
fig = px.scatter(Mainland_China, 
               x="Recovered", 
               y="Death",
               title='Chinese New Coronavirus Chart',
               template='plotly_dark', 
               log_x= False, 
               color='Province/State', 
               animation_frame= "Last Update",
               animation_group ="Province/State",
               hover_name='Province/State', 
               range_x = [-50,400], 
               range_y = [-50,400],
               size='Confirmed',
               size_max=50,
               labels = dict(Death='Death',
                             Recovered = 'Recovered'))
py.iplot(fig, filename='corona')

## 전 세계 비교 그래프

In [16]:
time_list = list(set(df['Last Update']))
time_list = pd.DataFrame(time_list, columns=['Last Update']).sort_values('Last Update')
time_list.index = range(len(time_list))

total_list=world_list+china_list+us_list+australia_list+canada_list

for i in total_list:
    i=i.replace(' ','_')
    time_list=pd.merge(time_list, globals()[i][['Last Update', 'Confirmed']], how='left', on='Last Update')
time_list.columns=['Last Update']+total_list
time_list=time_list.fillna(0)

df_world = time_list[['Last Update']+world_list]
df_china = time_list[['Last Update']+china_list]
df_china2 = time_list[['Last Update']+china_list2]
df_us = time_list[['Last Update']+us_list]
df_australia = time_list[['Last Update']+australia_list]
df_canada = time_list[['Last Update']+canada_list]

## 전 세계

In [44]:
data = df_world.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=world_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed by country',
            template='plotly_dark')

py.iplot(fig, filename='corona_world')

## 중국

In [45]:
data = df_china.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=china_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (China)',
            template='plotly_dark')

py.iplot(fig, filename='corona_china')

## 중국(후베이 제외)

In [46]:
data = df_china2.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=china_list2)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (China without Hubei)',
            template='plotly_dark'
           )

py.iplot(fig, filename='corona_china_without_hubei')

## 미국

In [47]:
data = df_us.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=us_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (US)',
            template='plotly_dark'
           )
py.iplot(fig, filename='corona_us')

## 호주

In [48]:
data = df_australia.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=australia_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (Australia)',
            template='plotly_dark'
           )
py.iplot(fig, filename='corona_australia')

## 캐나다

In [49]:
data = df_canada.copy()
data_melt=data.melt(id_vars='Last Update', value_vars=canada_list)
fig=px.line(data_melt, 
            x='Last Update', 
            y='value', 
            color='variable', 
            title='Number of virus confirmed (Canada)',
            template='plotly_dark'
           )
py.iplot(fig, filename='corona_canada')

## 개별 국가 시각화

## World

In [15]:
# for i in world_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## Mainland China

In [ ]:
# for i in china_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## US

In [ ]:
# for i in us_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## Australia

In [ ]:
# for i in australia_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()

## Canada

In [ ]:
# for i in canada_list:
#     print(i)
#     data = globals()[i.replace(' ','_')].copy()
#     data_melt = data.melt(id_vars='Last Update', value_vars=['Death', 'Recovered'])
#     data_melt2 = data.melt(id_vars='Last Update', value_vars=['D/C', 'R/C'])
    
#     fig = px.line(data, x="Last Update", y="Confirmed", title='Confirmed')
#     fig2= px.line(data_melt, x="Last Update", y="value", color='variable',title='Death and Recovered')
#     fig3= px.line(data_melt2, x="Last Update", y="value", color='variable',title='D/C and R/C')

#     fig.show()
#     fig2.show()
#     fig3.show()